In [ ]:
#DeepLSD model is taken from: https://github.com/cvg/DeepLSD

import os
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as lsc
import torch

from deeplsd.utils.tensor import batch_to_device
from deeplsd.models.deeplsd_inference import DeepLSD
from deeplsd.geometry.viz_2d import plot_images, plot_lines

In [2]:

# Model config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conf = {
    'detect_lines': True,  # Whether to detect lines or only DF/AF
    'line_detection_params': {
        'merge': False,  # Whether to merge close-by lines
        'filtering': 'strict',  # Whether to filter out lines based on the DF/AF. Use 'strict' to get an even stricter filtering
        'grad_thresh': 3,
        'grad_nfa': True,  # If True, use the image gradient and the NFA score of LSD to further threshold lines. We recommand using it for easy images, but to turn it off for challenging images (e.g. night, foggy, blurry images)
    }
}

# Load the model
ckpt = './deeplsd_md.tar'
ckpt = torch.load(str(ckpt), map_location='cpu')
net = DeepLSD(conf)
net.load_state_dict(ckpt['model'])
net = net.to(device).eval()

In [4]:
from PIL import Image, ImageDraw, ImageFilter

def detect_scalebar(path_image, thresh=0.5, isPlot=False):
    """
    Detects all the line segments using DeepLSD algorithm

    Args:
        path_image (str): path of the image to analyse
        thresh (float): angle threshold to determine if a segment is horizontal or not: it must in between 180+- thresh or 0+-thresh degrees
        isPlot (bool): boolean to plot or not the image

    Returns:
        pred_SB (numpy array): 4 elements numpy array containing the [x1, y1, x2, y2] coordinates of the detected bar
        indice_scalebar(int): indice of the line segment that is considered to be the scale bar
    """
    

    # Load an image
    img = cv2.imread(path_image)[:, :, ::-1]
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Detect (and optionally refine) the lines
    inputs = {'image': torch.tensor(gray_img, dtype=torch.float, device=device)[None, None] / 255.}
    with torch.no_grad():
        out = net(inputs)
        
        pred_lines = out['lines'][0]

    max = 0

    s = pred_lines.shape
    for i in range(s[0]):

        vector = pred_lines[i][0] - pred_lines[i][1]
        
        # Calculate the angle in radians
        angle_radians = np.arctan2(vector[1], vector[0])
        
        # Convert to degrees
        angle_degrees = np.degrees(angle_radians)

        if abs(angle_degrees) < thresh or abs(angle_degrees) > 180-thresh:
            v = pred_lines[i][1] - pred_lines[i][0] 
            distance = np.linalg.norm(v)
            if distance > max:
                max = distance
                pred_SB = pred_lines[i]
                indice_scalebar = i

    x1 = np.min([pred_SB[0][0],pred_SB[1][0]])
    y1 = np.mean([pred_SB[0][1],pred_SB[1][1]])

    x2 = np.max([pred_SB[0][0],pred_SB[1][0]])
    y2 = y1

    pred_SB =np.array([x1, y1, x2, y2])
    if isPlot:  
        shape = [(x1, y1),(x2, y2+5)] 
        Im_color = Image.open(path_image)
        img1 = ImageDraw.Draw(Im_color) 
        img1.rectangle(shape, fill="green", outline ="green")
        Im_color.show() 

    return pred_SB, indice_scalebar


In [8]:
import pandas as pd
import os 
import csv

test_folder = "./thorax_dataset/images/test/"
test_folder = "/Users/vivien.gaillet/Library/CloudStorage/OneDrive-NordAngliaEducation/NAE - Files/Downloads/original/"

csv_file_name = "thorax_only.csv"

#create new directory
new_dir_location = "./thorax_dataset/images/"
directory_name = "results_scale_bar/"
path_directory_save = os.path.join("./thorax_dataset/images/", directory_name)

# Create the directory if it doesn't exist
if not os.path.exists(path_directory_save):
    os.makedirs(path_directory_save)

#create saving files 

path_detected_bounding_box_csv = "./result_scale_bar_bounding_box.csv"

with open(path_detected_bounding_box_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ant", "x1_scalebar","y1_scalebar", "x2_scalebar", "y2_scalebar"])

df = pd.read_csv(csv_file_name)
df.head()

# List all image file names inside the folder
image_files = set(os.listdir(test_folder))

# Filter the DataFrame to retain only rows with filenames in the folder
filtered_df = df[df['ant'].isin(image_files)]
filtered_df.head()


,ant,x1_lmk,y1_lmk,x2_lmk,y2_lmk
0,anic32-900190-1_p_1.jpg,190.603985,199.479713,407.313962,328.349297
1,antweb1008066_p_1.jpg,175.484684,109.914697,397.234428,170.986006
2,antweb1008086_p_1.jpg,174.224742,117.000933,384.635011,197.457880
3,antweb1038001_p_1.jpg,137.686432,91.142189,388.414836,215.302585
4,antweb1038003_p_1.jpg,107.447831,71.837948,287.619498,114.624509


In [ ]:
"""
Detects the scale bars for all the images in the testing set,
and store the images of the detected scaler bar in "./thorax_dataset/images/results_scale_bar/"
and the bounding box pixel values in "./result_scale_bar_bounding_box.csv"
"""
for i, img in enumerate(filtered_df.iloc[:, 0]):            
    path = test_folder + img
    print(path)
    scalebar, i = detect_scalebar(path, thresh=0.5)
    print(scalebar)
    
    Im_color = Image.open(path)
    img1 = ImageDraw.Draw(Im_color) 
    t = 10
    shape = [(scalebar[0], scalebar[1]-t),(scalebar[0]+t, scalebar[1])] 
    img1.rectangle(shape, fill="red", outline ="red")

    shape = [(scalebar[2]-t, scalebar[3]+10),(scalebar[2], scalebar[3]+10+t)] 
    img1.rectangle(shape, fill="red", outline ="red")

    Im_color.save(path_directory_save + "scale_bar_" + img)
    Im_color.close() 

    t = 2
    new_row_csv = [img, scalebar[0],scalebar[1]-t,scalebar[2],scalebar[3]+t]
    print(new_row_csv)
    with open(path_detected_bounding_box_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Write the row
        writer.writerow(new_row_csv)


/Users/vivien.gaillet/Library/CloudStorage/OneDrive-NordAngliaEducation/NAE - Files/Downloads/original/anic32-900190-1_p_1.jpg
[213.99808 588.8345  460.00162 588.8345 ]
['anic32-900190-1_p_1.jpg', 213.99808, 586.83447265625, 460.00162, 590.83447265625]
/Users/vivien.gaillet/Library/CloudStorage/OneDrive-NordAngliaEducation/NAE - Files/Downloads/original/antweb1008066_p_1.jpg
[ 920.99927  781.77    1087.9993   781.77   ]
['antweb1008066_p_1.jpg', 920.99927, 779.77001953125, 1087.9993, 783.77001953125]
/Users/vivien.gaillet/Library/CloudStorage/OneDrive-NordAngliaEducation/NAE - Files/Downloads/original/antweb1008086_p_1.jpg


KeyboardInterrupt: 